In [1]:
import numpy as np
import torch
from pathlib import Path
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
from model import get_model
import warnings

warnings.filterwarnings("ignore")


In [2]:
# config seed for reproducible codes
SEED = 17167055
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
# load images
def loader(path):
    with open(path, "rb") as f:
        return Image.open(f).convert("RGB")


# stratified sampling
def stratified(df, col, n_smaples):
    n = min(n_smaples, df[col].value_counts().min())
    df_ = df.groupby(col).apply(lambda x: x.sample(n))
    df_.index = df_.index.droplevel(0)
    return df_


In [4]:
path = Path("../data/preprocess/")
data = [str(i) for i in list(path.glob("*.jpg"))]


In [5]:
df = pd.DataFrame()
df["filename"] = [i.split("\\")[-1] for i in data]
df["pixels"] = [loader(i) for i in data]
df["label"] = [int(i[0]) for i in df["filename"]]
df["id"] = ["-".join(i.split("_")[1:-2]) for i in df["filename"]]
df["var"] = [i.split(".")[0][-1] for i in df["filename"]]
df["ga"] = [i.split("_")[-2] for i in df["filename"]]


In [6]:
# remove all duplicates to make dataset more clean
df = df[df["var"] == "0"].reset_index(drop=True)


In [7]:
df


,filename,pixels,label,id,var,ga
0,0_057_21-5_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,057,0,21-5
1,0_080_21-3_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,080,0,21-3
2,0_082_21-6_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,082,0,21-6
3,0_087_21-2_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,087,0,21-2
4,0_096_21-6_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,096,0,21-6
...,...,...,...,...,...,...
85,1_JessTay-Sample8_21-4_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,JessTay-Sample8,0,21-4
86,1_May-IUGR2_21-3_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,May-IUGR2,0,21-3
87,1_May-IUGR4_22-0_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,May-IUGR4,0,22-0
88,1_SN-Sample12_IUGR3_22-4_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,SN-Sample12-IUGR3,0,22-4


In [8]:
df_duplicated = df[df.duplicated("id", keep=False)]
df_unique = df.drop(df_duplicated.index)
df_duplicated = df_duplicated.reset_index(drop=True)
df_unique = df_unique.reset_index(drop=True)

# test set
df_test = stratified(df_unique, "label", 5)
df_unique = df_unique.drop(df_test.index).reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


In [9]:
df_cv = pd.concat([df_duplicated, df_unique], ignore_index=True)


In [10]:
df_cv


,filename,pixels,label,id,var,ga
0,0_JessTay-Sample3_20-0_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,JessTay-Sample3,0,20-0
1,0_JessTay-Sample3_23-2_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,JessTay-Sample3,0,23-2
2,0_057_21-5_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,057,0,21-5
3,0_082_21-6_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,082,0,21-6
4,0_087_21-2_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,087,0,21-2
...,...,...,...,...,...,...
75,1_JessTay-Sample8_21-4_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,JessTay-Sample8,0,21-4
76,1_May-IUGR2_21-3_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,May-IUGR2,0,21-3
77,1_May-IUGR4_22-0_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,May-IUGR4,0,22-0
78,1_SN-Sample12_IUGR3_22-4_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,SN-Sample12-IUGR3,0,22-4


In [11]:
df_test


,filename,pixels,label,id,var,ga
0,0_172_22-0_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,172,0,22-0
1,0_May-N8_21-4_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,May-N8,0,21-4
2,0_143_22-1_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,143,0,22-1
3,0_JessLow-N6_22-2_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,JessLow-N6,0,22-2
4,0_080_21-3_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,0,080,0,21-3
5,1_169_22-1_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,169,0,22-1
6,1_161_21-0_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,161,0,21-0
7,1_177_22-3_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,177,0,22-3
8,1_188_S3_28-5_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,188-S3,0,28-5
9,1_JessTay-Sample6_22-5_0.jpg,<PIL.Image.Image image mode=RGB size=448x112 a...,1,JessTay-Sample6,0,22-5


In [12]:
scv = StratifiedKFold(n_splits=8, shuffle=True, random_state=SEED)
x_cv = df_cv.drop(columns=["label"])
y_cv = df_cv["label"]
cv_set = {
    "train": [],
    "val": [],
}
for idx, (train_index, val_index) in enumerate(scv.split(x_cv, y_cv)):
    # print(f"Split {idx}")
    # print(f"Total train: {len(train_index)}")
    # print(f"Total val: {len(val_index)}")
    cv_set["train"].append(df_cv.loc[train_index])
    cv_set["val"].append(df_cv.loc[val_index])
    # print("Val label:", cv_set["val"][idx]["label"].tolist())
    # print("\n")


In [13]:
transform = {
    "train": A.Compose(
        [
            A.CLAHE(p=0.8),
            A.RandomCrop(112, 112),
            A.Normalize(
                mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0
            ),
            ToTensorV2(),
        ]
    ),
    "test": A.Compose(
        [
            A.Normalize(
                mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0], max_pixel_value=255.0
            ),
            ToTensorV2(),
        ]
    ),
}


In [14]:
class DopplerDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.img = self.df["pixels"]
        self.lbl = self.df["label"]
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = self.img[idx]
        lbl = self.lbl[idx]
        if self.transform:
            img = self.transform(image=np.array(img))["image"]
        return img, lbl

In [15]:
def train(epoch, loader, net, model_name, split):
    net.train()
    totalLoss = 0
    correct = 0
    total = 0
    pbar = tqdm(loader, desc=f"Training split {split} {model_name}: {epoch+1}/{n_epoch}")
    for X, y in pbar:
        X, y = X.to(DEVICE), y.to(DEVICE)
        optimizer.zero_grad()
        out = net(X)
        y = y.unsqueeze(dim=1).float()
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        totalLoss += loss.item()
        pred = (torch.sigmoid(out) > 0.5).float()
        total += y.size(0)
        correct += pred.eq(y).cpu().sum()
        pbar.set_postfix({"loss": (totalLoss), "acc": (100.0 * correct / total).item()})
    acc = (100.0 * correct / total).item()
    return acc, totalLoss

In [16]:
def evaluate_val(epoch, loader, net, model_name, split):
    net.eval()
    totalLoss = 0
    correct = 0
    total = 2
    pbar = tqdm(loader, desc=f"Evaluating split {split} {model_name}: {epoch+1}/{n_epoch}")
    with torch.no_grad():
        for X, y in pbar:
            X, y = X.to(DEVICE), y.to(DEVICE)
            out = net(X)
            y = y.unsqueeze(dim=1).float()
            loss = criterion(out, y)
            totalLoss += loss.item()
            pred = (torch.sigmoid(out) > 0.5).float()
            # total += y.size(0)
            weight = 1/val_counts[1] if y.item()==1 else 1/val_counts[0]
            correct += pred.eq(y).cpu().sum()*weight
            pbar.set_postfix(
                {"loss": (totalLoss), "acc": (100.0 * correct / total).item()}
            )
    acc = (100.0 * correct / total).item()
    return acc, totalLoss

def evaluate_test(epoch, loader, net, model_name, split):
    net.eval()
    totalLoss = 0
    correct = 0
    total = 2
    pbar = tqdm(loader, desc=f"Evaluating split {split} {model_name}: {epoch+1}/{n_epoch}")
    with torch.no_grad():
        for X, y in pbar:
            X, y = X.to(DEVICE), y.to(DEVICE)
            out = net(X)
            y = y.unsqueeze(dim=1).float()
            loss = criterion(out, y)
            totalLoss += loss.item()
            pred = (torch.sigmoid(out) > 0.5).float()
            # total += y.size(0)
            weight = 1/test_counts[1] if y.item()==1 else 1/test_counts[0]
            correct += pred.eq(y).cpu().sum()*weight
            pbar.set_postfix(
                {"loss": (totalLoss), "acc": (100.0 * correct / total).item()}
            )
    acc = (100.0 * correct / total).item()
    return acc, totalLoss

# def evaluate_test(epoch, loader, net, model_name, split):
#     net.eval()
#     totalLoss = 0
#     correct = 0
#     total = 0
#     pbar = tqdm(loader, desc=f"Evaluating split {split} {model_name}: {epoch+1}/{n_epoch}")
#     with torch.no_grad():
#         for X, y in pbar:
#             X, y = X.to(DEVICE), y.to(DEVICE)
#             out = net(X)
#             y = y.unsqueeze(dim=1).float()
#             loss = criterion(out, y)
#             totalLoss += loss.item()
#             pred = (torch.sigmoid(out) > 0.5).float()
#             total += y.size(0)
#             correct += pred.eq(y).cpu().sum()
#             pbar.set_postfix(
#                 {"loss": (totalLoss), "acc": (100.0 * correct / total).item()}
#             )
#     acc = (100.0 * correct / total).item()
#     return acc, totalLoss

[0 0 0 1 1]
weight of 0 is 1/3
weight of 1 is 1/2
total weight = 2+3=5
[0 0 0 0 1]
acc = 1.5/2=50

In [17]:
testset = DopplerDataset(df_test, transform=transform["test"])
testLoader = DataLoader(testset, batch_size=1, shuffle=False)
test_counts = df_test["label"].value_counts()

In [18]:
results = dict(
    (
        splits,
        dict(
            (metrics, list())
            for metrics in [
                "train_acc",
                "train_loss",
                "val_acc",
                "val_loss",
                "test_acc",
                "test_loss",
            ]
        ),
    )
    for splits in range(8)
)
for split in range(len(cv_set["val"])):
    df_train = cv_set["train"][split].reset_index(drop=True)
    df_val = cv_set["val"][split].reset_index(drop=True)
    trainset = DopplerDataset(df_train, transform=transform["train"])
    valset = DopplerDataset(df_val, transform=transform["test"])
    model_name = "densenet121-pretrained-adam"
    model = get_model(model_name).to(DEVICE)
    lr = 1e-4
    n_epoch = 25
    bs = 16
    # filename = f"{model_name}-{lr}lr-{bs}bs-{n_epoch}e-split{split}-loss-weighted"
    filename = f"{model_name}-{lr}lr-{bs}bs-{n_epoch}e-split{split}-loss-normal"
    train_counts = df_train["label"].value_counts()
    val_counts = df_val["label"].value_counts()
    class_weights = list(1 / train_counts)
    sample_weights = [0] * len(df_train)
    for idx, row in df_train.iterrows():
        class_weight = class_weights[row["label"]]
        sample_weights[idx] = class_weight
    sampler = WeightedRandomSampler(
        sample_weights, num_samples=len(sample_weights), replacement=True
    )
    trainLoader = DataLoader(trainset, batch_size=bs, sampler=sampler)
    valLoader = DataLoader(valset, batch_size=1, shuffle=False)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(train_counts[1]/train_counts[0]))
    criterion = nn.BCEWithLogitsLoss()
    best_acc = 60.0
    for epoch in range(n_epoch):
        train_acc, train_loss = train(epoch, trainLoader, model, model_name, split)
        val_acc, val_loss = evaluate_val(epoch, valLoader, model, model_name, split)
        test_acc, test_loss = evaluate_test(epoch, testLoader, model, model_name, split)
        results[split]["train_acc"].append(train_acc)
        results[split]["train_loss"].append(train_loss)
        results[split]["val_acc"].append(val_acc)
        results[split]["val_loss"].append(val_loss)
        results[split]["test_acc"].append(test_acc)
        results[split]["test_loss"].append(test_loss)
        if val_acc > best_acc:
            best_acc = val_acc
            print(f"Best val {best_acc}, saving...")
            # torch.save(model, f"../model/{filename}-{best_acc}val-{epoch}e.pth")


Training split 0 densenet121-pretrained-adam: 1/25: 100%|██████████| 5/5 [00:04<00:00,  1.21it/s, loss=3.55, acc=58.6]
Evaluating split 0 densenet121-pretrained-adam: 1/25: 100%|██████████| 10/10 [00:00<00:00, 25.83it/s, loss=6.43, acc=44.4]
Evaluating split 0 densenet121-pretrained-adam: 1/25: 100%|██████████| 10/10 [00:00<00:00, 27.82it/s, loss=6.89, acc=70]
Training split 0 densenet121-pretrained-adam: 2/25: 100%|██████████| 5/5 [00:00<00:00,  6.01it/s, loss=3.28, acc=64.3]
Evaluating split 0 densenet121-pretrained-adam: 2/25: 100%|██████████| 10/10 [00:00<00:00, 28.08it/s, loss=6.45, acc=33.3]
Evaluating split 0 densenet121-pretrained-adam: 2/25: 100%|██████████| 10/10 [00:00<00:00, 25.18it/s, loss=6.4, acc=70]
Training split 0 densenet121-pretrained-adam: 3/25: 100%|██████████| 5/5 [00:00<00:00,  5.21it/s, loss=3.1, acc=64.3]  
Evaluating split 0 densenet121-pretrained-adam: 3/25: 100%|██████████| 10/10 [00:00<00:00, 26.45it/s, loss=6.76, acc=83.3]
Evaluating split 0 densenet121-p

Best val 83.33333587646484, saving...


Training split 0 densenet121-pretrained-adam: 4/25: 100%|██████████| 5/5 [00:00<00:00,  5.73it/s, loss=2.73, acc=72.9]
Evaluating split 0 densenet121-pretrained-adam: 4/25: 100%|██████████| 10/10 [00:00<00:00, 26.04it/s, loss=7, acc=22.2]  
Evaluating split 0 densenet121-pretrained-adam: 4/25: 100%|██████████| 10/10 [00:00<00:00, 25.25it/s, loss=5.9, acc=80]
Training split 0 densenet121-pretrained-adam: 5/25: 100%|██████████| 5/5 [00:00<00:00,  5.99it/s, loss=2.75, acc=74.3] 
Evaluating split 0 densenet121-pretrained-adam: 5/25: 100%|██████████| 10/10 [00:00<00:00, 26.52it/s, loss=6.8, acc=33.3]
Evaluating split 0 densenet121-pretrained-adam: 5/25: 100%|██████████| 10/10 [00:00<00:00, 25.97it/s, loss=5.81, acc=80]
Training split 0 densenet121-pretrained-adam: 6/25: 100%|██████████| 5/5 [00:00<00:00,  5.92it/s, loss=2.31, acc=75.7]
Evaluating split 0 densenet121-pretrained-adam: 6/25: 100%|██████████| 10/10 [00:00<00:00, 25.50it/s, loss=6.34, acc=38.9]
Evaluating split 0 densenet121-pre

Best val 68.75, saving...


Training split 2 densenet121-pretrained-adam: 11/25: 100%|██████████| 5/5 [00:00<00:00,  6.25it/s, loss=2.14, acc=80]   
Evaluating split 2 densenet121-pretrained-adam: 11/25: 100%|██████████| 10/10 [00:00<00:00, 23.36it/s, loss=5.25, acc=75] 
Evaluating split 2 densenet121-pretrained-adam: 11/25: 100%|██████████| 10/10 [00:00<00:00, 22.83it/s, loss=6.78, acc=60]


...

Training split 5 densenet121-pretrained-adam: 4/25: 100%|██████████| 5/5 [00:00<00:00,  6.06it/s, loss=2.9, acc=78.6]  
Evaluating split 5 densenet121-pretrained-adam: 4/25: 100%|██████████| 10/10 [00:00<00:00, 28.73it/s, loss=5.49, acc=68.8]
Evaluating split 5 densenet121-pretrained-adam: 4/25: 100%|██████████| 10/10 [00:00<00:00, 26.17it/s, loss=7.24, acc=40]
Training split 5 densenet121-pretrained-adam: 5/25: 100%|██████████| 5/5 [00:00<00:00,  5.63it/s, loss=2.51, acc=80]  
Evaluating split 5 densenet121-pretrained-adam: 5/25: 100%|██████████| 10/10 [00:00<00:00, 26.66it/s, loss=5.37, acc=50] 
Evaluating split 5 densenet121-pretrained-adam: 5/25: 100%|██████████| 10/10 [00:00<00:00, 24.03it/s, loss=7.53, acc=50]
Training split 5 densenet121-pretrained-adam: 6/25: 100%|██████████| 5/5 [00:00<00:00,  6.04it/s, loss=2.4, acc=74.3]  
Evaluating split 5 densenet121-pretrained-adam: 6/25: 100%|██████████| 10/10 [00:00<00:00, 25.97it/s, loss=5.62, acc=50] 
Evaluating split 5 densenet121-p

Best val 75.0, saving...


Training split 5 densenet121-pretrained-adam: 12/25: 100%|██████████| 5/5 [00:00<00:00,  6.19it/s, loss=1.31, acc=95.7] 
Evaluating split 5 densenet121-pretrained-adam: 12/25: 100%|██████████| 10/10 [00:00<00:00, 24.75it/s, loss=6, acc=75]    
Evaluating split 5 densenet121-pretrained-adam: 12/25: 100%|██████████| 10/10 [00:00<00:00, 29.32it/s, loss=7.27, acc=50]
Training split 5 densenet121-pretrained-adam: 13/25: 100%|██████████| 5/5 [00:00<00:00,  5.65it/s, loss=1.6, acc=87.1]  
Evaluating split 5 densenet121-pretrained-adam: 13/25: 100%|██████████| 10/10 [00:00<00:00, 26.45it/s, loss=6.19, acc=75] 
Evaluating split 5 densenet121-pretrained-adam: 13/25: 100%|██████████| 10/10 [00:00<00:00, 26.45it/s, loss=7.06, acc=30]
Training split 5 densenet121-pretrained-adam: 14/25: 100%|██████████| 5/5 [00:00<00:00,  5.58it/s, loss=0.914, acc=91.4]
Evaluating split 5 densenet121-pretrained-adam: 14/25: 100%|██████████| 10/10 [00:00<00:00, 26.80it/s, loss=6.7, acc=37.5]
Evaluating split 5 dense

Best val 75.0, saving...


Training split 6 densenet121-pretrained-adam: 10/25: 100%|██████████| 5/5 [00:00<00:00,  5.67it/s, loss=1.84, acc=84.3] 
Evaluating split 6 densenet121-pretrained-adam: 10/25: 100%|██████████| 10/10 [00:00<00:00, 26.04it/s, loss=8.8, acc=50]
Evaluating split 6 densenet121-pretrained-adam: 10/25: 100%|██████████| 10/10 [00:00<00:00, 27.17it/s, loss=6.73, acc=50]
Training split 6 densenet121-pretrained-adam: 11/25: 100%|██████████| 5/5 [00:00<00:00,  5.97it/s, loss=2.21, acc=80]   
Evaluating split 6 densenet121-pretrained-adam: 11/25: 100%|██████████| 10/10 [00:00<00:00, 27.77it/s, loss=9.87, acc=50]
Evaluating split 6 densenet121-pretrained-adam: 11/25: 100%|██████████| 10/10 [00:00<00:00, 26.45it/s, loss=6.84, acc=40]
Training split 6 densenet121-pretrained-adam: 12/25: 100%|██████████| 5/5 [00:00<00:00,  5.85it/s, loss=1.43, acc=91.4] 
Evaluating split 6 densenet121-pretrained-adam: 12/25: 100%|██████████| 10/10 [00:00<00:00, 26.17it/s, loss=9.37, acc=56.2]
Evaluating split 6 densene

In [19]:
import pickle
a_file = open(f"../result_cv/{filename}.pkl", "wb")
pickle.dump(results, a_file)
a_file.close()

In [20]:
# 0-8, 1-2

(7/9+0/1)/2*100

38.88888888888889